In [8]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import os
from pathlib import Path
from keras.applications import vgg19
from keras.preprocessing import image
from keras.models import model_from_json
import cv2

In [9]:
# Construct an relative path
# os.path.join(os.getcwd(), "training-data", ) --- To do later

In [10]:
# Construct the image path using the Path module
image_path = Path("./training-data/dataset1/known_images")

In [11]:
# Create array for the images and labels
images = []
labels = []

In [21]:
# Set a target size for the imagers
target_size = (391, 488)

In [22]:
# Load all the normal images and create labels for them
for img in image_path.glob("normal*.png"):
    # Load images from the disk
    img = image.image_utils.load_img(img, target_size=target_size)

    # Convert the image to numpy array
    image_array = image.image_utils.img_to_array(img)

    # Add the image to a list of images
    images.append(image_array)

    # For each dog image, the label should 1
    labels.append(0)

In [24]:
# Load all the opacity images and create labels for them
for img in image_path.glob("sick*.png"):
    # Load images from the disk
    img = image.image_utils.load_img(img, target_size=target_size)

    # Convert the image to numpy array
    image_array = image.image_utils.img_to_array(img)

    # Add the image to a list of images
    images.append(image_array)

    # For each dog image, the label should 1
    labels.append(1)

In [25]:
# Create a numpy array of all the images
X_train = np.array(images)

# Create a numpy array of all the labels
Y_train = np.array(labels)

In [28]:
X_train.shape

(715, 391, 488, 3)

In [26]:
# Normalize image data from (0 - 255) to (0 - 1)
X_train = vgg19.preprocess_input(X_train);

In [29]:
# Load a pre-trained neural network to use as feature extractor
pre_trained_neural_network = vgg19.VGG19(weights="imagenet", include_top=False, input_shape=(391, 488, 3))

In [30]:
# Extract features for each image
features_X = pre_trained_neural_network.predict(X_train)

2023-05-02 21:14:42.269476: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


23/23 [==============================] - 338s 15s/step


In [31]:
# Save the array of extracted features to a file
joblib.dump(features_X, "X_train.dat")

['X_train.dat']

In [32]:
# save the matching array of labels to the file
joblib.dump(Y_train, "Y_train.dat")

['Y_train.dat']

In [33]:
# Load the features and labels from teh .dat files
X_train = joblib.load("X_train.dat")
Y_train = joblib.load("Y_train.dat")

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

In [38]:
# Create a model and add layers
model = Sequential()

# Flatten the input from the vgg19 model convolution nn
model.add(Flatten(input_shape=X_train.shape[1:]))

model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

In [39]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [42]:
# Train the model
model.fit(
    X_train,
    Y_train,
    epochs=10,
    batch_size=32,
    shuffle=True
)

Epoch 1/10
23/23 [==============================] - 1s 40ms/step - loss: 4.5736e-08 - accuracy: 1.0000
Epoch 2/10
23/23 [==============================] - 1s 37ms/step - loss: 0.0849 - accuracy: 0.9930
Epoch 3/10
23/23 [==============================] - 1s 36ms/step - loss: 0.1162 - accuracy: 0.9888
Epoch 4/10
23/23 [==============================] - 1s 36ms/step - loss: 0.2917 - accuracy: 0.9916
Epoch 5/10
23/23 [==============================] - 1s 41ms/step - loss: 3.6329e-07 - accuracy: 1.0000
Epoch 6/10
23/23 [==============================] - 1s 36ms/step - loss: 1.0018e-08 - accuracy: 1.0000
Epoch 7/10
23/23 [==============================] - 1s 36ms/step - loss: 0.0486 - accuracy: 0.9986
Epoch 8/10
23/23 [==============================] - 1s 36ms/step - loss: 0.2639 - accuracy: 0.9902
Epoch 9/10
23/23 [==============================] - 1s 35ms/step - loss: 0.2909 - accuracy: 0.9902
Epoch 10/10
23/23 [==============================] - 1s 37ms/step - loss: 0.2337 - accuracy: 0.99

In [44]:
# Save the neural network structure
model_structure = model.to_json()
f = Path("./models/transfer-learning/model_structure.json")
f.write_text(model_structure)

1464

In [45]:
# Save the neural network trained weights
model.save_weights("./models/transfer-learning/model_weights.h5")

In [46]:
# Load the json file the contains the model structure
f = Path("./models/transfer-learning/model_structure.json")
model_structure = f.read_text()

In [47]:
# Create the keras model object from the model_structure json data
model = model_from_json(model_structure)

In [48]:
# Re-load the models trained weigths
model.load_weights("./models/transfer-learning/model_weights.h5")